In [ ]:
import pandas as pd
import plotnine as p9
import yaml

In [ ]:
COLOR_SCALE = ["#00577B", "#24A899"]

In [ ]:
with open("data/portfolios.yml", "r") as fp:
    portfolios = yaml.load(fp, Loader=yaml.SafeLoader)
portfolios

In [ ]:
data = []

for portfolio in portfolios:
    id = portfolio["name"]
    summary = pd.read_excel(f"output/reports/{id}.xlsx", sheet_name="Summary", header=None)
    df = pd.DataFrame([
        dict(portfolio=id, profile="flat", metric="CO2", value=summary.iloc[0,1]),
        dict(portfolio=id, profile="tracking", metric="CO2", value=summary.iloc[1,1]),
        dict(portfolio=id, profile="flat", metric="cost", value=summary.iloc[4,1]),
        dict(portfolio=id, profile="tracking", metric="cost", value=summary.iloc[5,1]),
    ])
    data.append(df)

data = pd.concat(data, axis=0)

In [ ]:
fig = (
    p9.ggplot(p9.aes(x="portfolio", fill="profile", y="value"), data.query("metric == 'CO2'").assign(value=lambda df: df.value / 1000.0)) +
    p9.geom_bar(stat="identity", position="dodge") +
    p9.labs(y="Emission (kg CO2/MWh)", title="Average CO2 emission factor") +
    p9.theme(
        axis_title_x=p9.element_blank(),
        figure_size=(7, 5)
    ) +
    p9.scale_fill_manual(values=COLOR_SCALE)
)
fig.save("output/plots/summary_barplot_co2.png", dpi=150)

In [ ]:
fig = (
    p9.ggplot(p9.aes(x="portfolio", fill="profile", y="value"), data.query("metric == 'cost'")) +
    p9.geom_bar(stat="identity", position="dodge") +
    p9.labs(y="Cost (EUR/MWh)", title="Average EPEX day-ahead price") +
    p9.theme(
        axis_title_x=p9.element_blank(),
        figure_size=(7, 5)
    ) +
    p9.scale_fill_manual(values=COLOR_SCALE)
)
fig.save("output/plots/summary_barplot_cost.png", dpi=150)

In [ ]:
for portfolio in portfolios:
    id = portfolio["name"]
    df = pd.read_excel(f"output/reports/{id}.xlsx", sheet_name="Data")
    fig = (
        p9.ggplot(p9.aes(x="profile", y="co2"), df) +
        p9.geom_point() +
        p9.geom_smooth(method="loess", span=0.5, color=COLOR_SCALE[1]) +
        p9.labs(x="Consumption (kWh)", y="CO2 emission factor (kg/MWh)", title=f"Correlation of consumption and emission for portfolio {id}") +
        p9.theme(figure_size=(6, 5))
    )
    fig.save(f"output/plots/correlation_co2_{id}.png", dpi=200)

In [ ]:
for portfolio in portfolios:
    id = portfolio["name"]
    df = pd.read_excel(f"output/reports/{id}.xlsx", sheet_name="Data")
    fig = (
        p9.ggplot(p9.aes(x="profile", y="price"), df) +
        p9.geom_point() +
        p9.geom_smooth(method="loess", span=0.5, color=COLOR_SCALE[1]) +
        p9.labs(x="Consumption (kWh)", y="Price (EUR/kWh)", title=f"Correlation of consumption and spot price for portfolio {id}") +
        p9.theme(figure_size=(6, 5))
    )
    fig.save(f"output/plots/correlation_price_{id}.png", dpi=200)